In [315]:
# conda activate ESE527
# Import the library
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import os
from medcat.vocab import Vocab
from medcat.cdb import CDB
from medcat.cat import CAT
from medcat.meta_cat import MetaCAT
import textwrap
from rich import print 
import os
import json
import re
import pickle
import time
from tqdm import tqdm, trange
from concurrent.futures import ThreadPoolExecutor
from collections import defaultdict
from collections import Counter


### Prepared data and model


In [316]:
# Load the vocab model we downloaded 
vocab = Vocab.load("medcat_models/vocab.dat")
# Load the cdb model you downloaded
cdb = CDB.load('medcat_models/cdb.dat') 

c:\Users\judyw\Desktop\ESE527\project\.conda\Lib\site-packages\pydantic\main.py:426: UserWarning: Pydantic serializer warnings:
  Expected `set[str]` but got `dict` with value `{}` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(
which may or may not work. If you experience any compatibility issues, please reinstall MedCAT
or download the compatible model.


In [317]:
# Run the medcat_status model
mc_status = MetaCAT.load("medcat_models/meta_Status")

c:\Users\judyw\Desktop\ESE527\project\.conda\Lib\site-packages\medcat\meta_cat.py:430: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  meta_cat.model.load_state_dict(torch.loa

In [318]:
# Initial the CAT
# Please run twice
cat = CAT(cdb=cdb, config=cdb.config, vocab=vocab, meta_cats=[mc_status])


Traceback (most recent call last):
  File "c:\Users\judyw\Desktop\ESE527\project\.conda\Lib\site-packages\medcat\pipe.py", line 52, in __init__
    self._nlp = self._init_nlp(config)
                ^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\judyw\Desktop\ESE527\project\.conda\Lib\site-packages\medcat\pipe.py", line 76, in _init_nlp
    return spacy.load(config.general.spacy_model, disable=config.general.spacy_disabled_components)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\judyw\Desktop\ESE527\project\.conda\Lib\site-packages\spacy\__init__.py", line 51, in load
    return util.load_model(
           ^^^^^^^^^^^^^^^^
  File "c:\Users\judyw\Desktop\ESE527\project\.conda\Lib\site-packages\spacy\util.py", line 472, in load_model
    raise IOError(Errors.E050.format(name=name))
OSError: [E050] Can't find model 'spacy_model'. It doesn't seem to be a Python package or a valid path to a data directory.


In [319]:
# Read the MIMIC-III data
# Read all of csv file
# Read the required csv file first
note_event_df = pd.read_csv('Data/NOTEEVENTS.csv', low_memory=False)
patients_df = pd.read_csv('Data/PATIENTS.csv', low_memory=False)
admissions_df = pd.read_csv('Data/ADMISSIONS.csv',  low_memory=False)


In [320]:
# Just a look of the the noteevent 
note_event_df.head()

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
0,174,22532,167853.0,2151-08-04,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2151-7-16**] Dischar...
1,175,13702,107527.0,2118-06-14,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2118-6-2**] Discharg...
2,176,13702,167118.0,2119-05-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2119-5-4**] D...
3,177,13702,196489.0,2124-08-18,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2124-7-21**] ...
4,178,26880,135453.0,2162-03-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2162-3-3**] D...


In [321]:
# Load the doc2text
"""
ROW ID as the key
Notes as the item
Target: Reduce the read the note.csv several times. We will
use the row id as the index to connect the other info
"""

# Convert the Noteevents to the pickle files
doc2text = {}
note_info = note_event_df[['ROW_ID', 'TEXT']]
for id, info in note_info.iterrows():
    row_id = str(info['ROW_ID'])
    doc2text[row_id] = {
        'TEXT': info['TEXT']
    }
    


In [322]:
# Save the dictionary into the pickle file doc2text
with open('doc2text.pickle', 'wb') as f:
    pickle.dump(doc2text, f)

size_note = len(doc2text)
# Check the size of the noteevents and the pickle file already generated
print(f"We already the doc2text file and we have {size_note} clinical notes.")

We already the doc2text file and we have 2083180 clinical notes.

In [323]:
# If we have a doc2text right now, we need to load it
with open('doc2text.pickle', 'rb') as f:
    doc2text = pickle.load(f)
print("Opened doc2text pickle file")

Opened doc2text pickle file

In [324]:
"""
construct the doc2info dictionary, the key should be the row id and the 
chartdate and subject id should be value
"""
doc2info = {}
info_df = note_event_df[['ROW_ID', 'SUBJECT_ID', 'CHARTDATE']]
info_df['CHARTDATE'] = pd.to_datetime(info_df['CHARTDATE'])  
for id, row in info_df.iterrows():
    row_id = str(row['ROW_ID'])
    doc2info[row_id] = {
        'SUBJECT_ID': row['SUBJECT_ID'],
        'CHARTDATE': row['CHARTDATE']
    }

with open('doc2info.pickle', 'wb') as f:
    pickle.dump(doc2info, f)
# Check for the first 3 dict
for doc_id, info in list(doc2info.items())[:3]:
    print(doc_id, "=>", info)
    print(type(doc_id))




C:\Users\judyw\AppData\Local\Temp\ipykernel_38868\461167509.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  info_df['CHARTDATE'] = pd.to_datetime(info_df['CHARTDATE'])


174 =>
{'SUBJECT_ID': 22532, 'CHARTDATE': Timestamp('2151-08-04 00:00:00')}

<class 'str'>

175 =>
{'SUBJECT_ID': 13702, 'CHARTDATE': Timestamp('2118-06-14 00:00:00')}

<class 'str'>

176 =>
{'SUBJECT_ID': 13702, 'CHARTDATE': Timestamp('2119-05-25 00:00:00')}

<class 'str'>

In [325]:
# Data cleaning method 
def data_clean(text):
    # Remove this format
    # Remove the token like this "[**Name**]"
    clean_step1 = re.sub(r'\[\*\*.*?\*\*\]', ' ', text)
    cleaned = re.sub(r'\s+', ' ', clean_step1)
    cleaned_redundency = re.sub("([^0-9])\\1{3,}", "\\1\\1\\1", cleaned)
    final_cleaned = cleaned_redundency.strip()
    return final_cleaned


In [326]:
# Call the clean function
for id, content in doc2text.items():
    content['TEXT'] = data_clean(content['TEXT'])



In [327]:
# Small check for the cleaned data
for doc_id, text in list(doc2text.items())[:1]:
    print(doc_id, "=>", text)
    print(len(text['TEXT']))

174 =>
{
    'TEXT': 'Admission Date: Discharge Date: Service: ADDENDUM: RADIOLOGIC STUDIES: Radiologic studies also 
included a chest CT, which confirmed cavitary lesions in the left lung apex consistent with infectious 
process/tuberculosis. This also moderate-sized left pleural effusion. HEAD CT: Head CT showed no intracranial 
hemorrhage or mass effect, but old infarction consistent with past medical history. ABDOMINAL CT: Abdominal CT 
showed lesions of T10 and sacrum most likely secondary to osteoporosis. These can be followed by repeat imaging as 
an outpatient. , M.D. Dictated By: MEDQUIST36 D: 12:11 T: 12:21 JOB#:'
}

603

In [328]:
def extract_medcat_entity(text):
    doc = cat.get_entities(text)
    entities_info = doc['entities'].values()
    
    return entities_info



In [329]:

def replace_text_cui(text, entities):
    """
    We should replace the medical concept with the cui (Target Types)
    Args:
    text (str): Original text from clinical notes.
    entities (list): List of entities extracted by MedCAT, each entity is a dict containing:
        - 'start': start index
        - 'end': end index
        - 'cui': concept CUI code
        - 'type': entity type (disorders, symptoms, findings, medications, etc.)
    
    Returns: 
        the text already replaced by cui
    """
    # Target Type 
    #target_types = {'disorders','symptoms', 'findings', 'medications'}
    # extend the range of the types we may need to use 
    target_types = {
    'disorder', 'finding', 'procedure', 
    'morphologic abnormality','medications','symptoms'
    }
    # Sort thhe enitities through the start index which may help us to avoid modify the position of 
    entities_sorted = sorted(entities, key=lambda e: e['start'], reverse=True)
    replaced_text = text
    for ent in entities_sorted:
        # Skip the type we do not expect
        if not any(t in target_types for t in ent['types']):
            continue
        # Append the text from the last index to next enitity
        start, end, cui = ent['start'], ent['end'], ent['cui']

        #breakpoint()
        #print(ent)
        # Consider if spacing is necessay 
        #print(f"Will be replaced by : '{replaced_text[start:end]}' -> '{cui}'")

        # Contains the text before the current entity (the concept mention we want to replace).
        prefix = replaced_text[:start]
      
        # Contains the text after the current entity (after the replaced part).
        suffix = replaced_text[end:]
        # Space management: add spacing around CUIs to avoid accidental merging of the text
        # (e.g., "wordCUI" becomes "word CUI").
        """
        functions: 
        1. .isalnum() returns True if it's alphanumeric (letter or number), and False if it's a punctuation mark or whitespace.
        Target: If the last character before the inserted CUI is alphanumeric, this means the word before would directly touch the inserted CUI code, creating something like "patienthasC0011849". 
        To avoid this, the condition inserts a space (' ').
        """
        space_before = ' ' if prefix and prefix[-1].isalnum() else ''
        space_after = ' ' if suffix and suffix[0].isalnum() else ''
        # Append any remaining text after the last entity
        replaced_text = prefix + space_before + cui + space_after + suffix
    # Join all parts into the final text
    return replaced_text

In [330]:
# Build the pt2info, key is the subject id
pt2info = {}

# Convert the dob and dod to time format
patients_df['DOB'] = pd.to_datetime(patients_df['DOB'])
patients_df['DOD'] = pd.to_datetime(patients_df['DOD'])

# Build a function to read the two df inf simutanously. 
def build_pt2info(pt2info, patients_df, admissions_df):
    for id, row in patients_df.iterrows():
        patients_id = str(row['SUBJECT_ID'])
        # Initial the gender 
        gender = 'Unknown'
        # get the gender 
        if row['GENDER'] == 'F':
            gender = 'Female'
        elif row['GENDER'] == 'M':
            gender = 'Male'
        
        # Date of Death & Date of Birth
        dod = row['DOD']
        dob = row['DOB']
        if pd.isna(row['DOD']):
            dod = None
        if pd.isna(row['DOB']):
            dob = None

        # Store the info into the 
        pt2info[patients_id] = {
            'Gender': gender,
            'DOD': dod,
            'DOB': dob
        }
    for id,row in admissions_df.iterrows():
        ethencity = 'Unknown'
        # pd.notna():  detects existing/ non-missing values in the dataframe. 
        if pd.notna(row['ETHNICITY']):
            ethencity = row['ETHNICITY']
        
        pt2info[str(row['SUBJECT_ID'])]['ETHNICITY'] = ethencity
    return pt2info


pt2info = build_pt2info(pt2info, patients_df, admissions_df)

# Save the dictionary into the pickle file doc2text
with open('pt2info.pickle', 'wb') as f:
    pickle.dump(pt2info, f)

        


In [331]:
# Define how to build patient time line
# load the pt2info first
with open('pt2info.pickle', 'rb') as f:
    pt2info = pickle.load(f)

In [332]:
# Call the replace the text context by using the replace the text with cui function
for doc_id, text in list(doc2text.items())[:3]:
    entities = extract_medcat_entity(text['TEXT'])
    #print("Before Updated CUI TEXT: ", text)
    text_cui = replace_text_cui(text['TEXT'], entities)
    doc2text[doc_id]['TEXT'] = text_cui
    #print("Updated CUI TEXT: ", doc2info[doc_id])


🧠 什么是 Bucketing（时间桶）？
Bucketing 是一种按照时间窗口（bucket）将概念进行聚合的技术。

在 FS2 中，它指的是：

将同一患者在同一天（1-day window）内提取到的多个概念，合并成一个“时间桶”中，只保留唯一的概念。
✅ Bucketing 的目的
去除重复：
医疗笔记中同一个概念可能会在一天内被反复提及（如 "diabetes" 在多个 notes 中出现），不利于模型训练。通过 bucketing，将这些重复概念只保留一次。

降噪与错误识别：
如果一个概念只在整条时间线上出现一次，很可能是命名实体识别（NER）或链接错误（Linking mistake），因此可被认为是“可疑概念”并剔除。

标准化时间表达：
它有助于统一每个时间点上的概念粒度，形成更稳定、可学习的事件单位。

pt2note → pt2timeline（按时间排序并插入标记）
      → filter_rare_concepts（去除低频CUI）
      → bucket_timeline（合并同一日笔记，生成结构化输入）
      

In [333]:
# Load the doc2info information
with open('doc2info.pickle', 'rb') as f:
    doc2info = pickle.load(f)
# Apply the bucketing in building patient timeline

# We should group patients info depend on subject id 

pt2note = defaultdict(list)  

for row_id, text in list(doc2text.items())[:200]:
    # Use the row id to find correlated 
    flag = doc2info.get(row_id)
    #print(f"row_id: {row_id}, flag: {flag}, type: {type(flag)}")

    # if we cannot find this id
    if not flag:
        continue # Go to the next one
    
    patient_subject_id = flag['SUBJECT_ID']
    time_stamp = flag['CHARTDATE']
    # Used updated text and entity
    entities = extract_medcat_entity(text['TEXT'])
    replaced_text = replace_text_cui(text['TEXT'], entities) 
    # Use the subject id as the patient id
    pt2note[patient_subject_id].append((time_stamp, replaced_text))


# Sort the patient events based on the chart date
for subject_id, notes in list(pt2note.items())[:200]:
    pt2note[subject_id] = sorted(notes, key=lambda x: x[0])
    



In [334]:
# Check point for grouping result
for subject_id, notes in list(pt2note.items())[:1]:
    print(subject_id)
    for ts, txt in notes:
        print(" -", ts.date(), "->", txt, "...")
        print(len(txt))
    print("—" * 40)

22532

- 2151-08-04 -> 32485007 Date: 75823008 Date: Service: ADDENDUM: RADIOLOGIC STUDIES: Radiologic studies also 
included a 169069000, which confirmed cavitary 52988006 in the left lung apex consistent with infectious 
process/56717001. This also moderate-sized left 60046008. 303653007: 303653007 showed no 1386000 or 274724004, but 
old 55641003 consistent with past medical history. ABDOMINAL CT: Abdominal CT showed 52988006 of T10 and sacrum 
most likely secondary to 64859006. These can be followed by repeat 363679005 as an outpatient. , M.D. Dictated By: 
MEDQUIST36 D: 12:11 T: 12:21 JOB#: ...

574

————————————————————————————————————————

In [335]:
# Convert the time sets into time stream 
def patient_timeline(pt2note):
    pt2timeline = {}

    """Change the structure of the pt2note"""
    # Every patients' note has been sorted by timestamp
    for subject_id, notes in pt2note.items():
        # This timeline variable will store each timeline for each patient
        timeline = []
        # Previous time stamp
        prev_time = None
        # Iterate each sorted notes based on the time difference
        for timestamp, text in notes:
            # if this not the first notes
            if prev_time is not None:
                skip = ""
                # Calculate the time seperation 
                sep = (timestamp - prev_time).days
                if sep > 1 and sep < 7:
                    skip = f"<{sep} days later>"
                elif sep > 7 and sep < 30:
                    skip = f"<{sep // 7 } weeks later>"
                elif sep == 1:
                    skip = f"<{sep} day later>"
                elif sep == 7:
                    skip = f"<{sep // 7 } week later>"
                elif sep == 30:
                    skip = f"<{sep // 30 } month later>"
                elif sep > 30 and sep < 365:
                    skip = f"<{sep // 30 } months later>"
                elif sep == 365:
                    skip = f"<{sep // 365 } year later>" 
                else:
                    skip = f"<{sep // 365 } years later>"    
                # Insert the time token
                timeline.append(skip)
            # Insert the current notes
            timeline.append(text) 
            prev_time = timestamp
            # Concate all timeline records into the dict. 
        pt2timeline[subject_id] = "\n".join(timeline)
    return pt2timeline



In [336]:
# First we have to count the frequency of the each cui in the repalced text; if this medical concept
# only appears 1 times, this may a mistake 

# Define the Filter function 
def global_frequency_check(pt2timeline):
    # Find the pattern
    cui_pattern = re.compile(r'\d{6,}') 
    global_counter = Counter()
    # Calculate the frequency
    for timeline_text in pt2timeline.values():
        cuis = cui_pattern.findall(timeline_text)
        global_counter.update(cuis)
    return global_counter

def filter_function(pt2timeline, global_counter):
    cui_pattern = re.compile(r'\d{6,}') 
    filtered_stream = {}
    pt2removed_cuis = {}
    for subject_id, notes in pt2timeline.items():
        all_cuis = cui_pattern.findall(notes)
        valid_cuis = set()
        removed_cui = set()
        # Check if cui freequency is larger then 2
        for cui in set(all_cuis):
            if global_counter[cui] >= 2:
                valid_cuis.add(cui)
            else:
                removed_cui.add(cui)
        
        def replace_rare_cuis(match):
            # Match.group(0) will return the object (string)
            cui = match.group(0)
            return cui if cui in valid_cuis else ""   
        
        cleaned_text = cui_pattern.sub(replace_rare_cuis, notes)
        filtered_stream[subject_id] = cleaned_text
        pt2removed_cuis[subject_id] = list(removed_cui)
        
    return filtered_stream, pt2removed_cuis  

        


In [337]:
# Call function covert sets to stream 
pt2timeline = patient_timeline(pt2note)
global_counts = global_frequency_check(pt2timeline)
cleaned_pt2timeline, removed_log = filter_function(pt2timeline,global_counts)

In [338]:
for subject_id, notes in list(cleaned_pt2timeline.items())[:5]:
    print(subject_id)
    print(notes)
    print(len(notes))
    print(len(removed_log))




22532

32485007 Date: 75823008 Date: Service: ADDENDUM: RADIOLOGIC STUDIES: Radiologic studies also included a 169069000, 
which confirmed cavitary 52988006 in the left lung apex consistent with infectious process/56717001. This also 
moderate-sized left 60046008. 303653007: 303653007 showed no 1386000 or 274724004, but old 55641003 consistent with
past medical history. ABDOMINAL CT: Abdominal CT showed 52988006 of T10 and sacrum most likely secondary to 
64859006. These can be followed by repeat 363679005 as an outpatient. , M.D. Dictated By: MEDQUIST36 D: 12:11 T: 
12:21 JOB#:

574

119

13702

...

119

20646

32485007 Date: 75823008 Date: Service: MEDICINE Allergies: Sulfonamides Attending: 33962009: 389086002 Major 
Surgical or Invasive Procedure: none History of Present 39104002: 82 yo F with 53741008, 42343007, HTN, recent PE (
), who presents from rehab with 389086002 and 267036007 despite Abx treatment for PNA x 3 days. The patient was in 
rehab after being 183665006 from here for PE. She was scheduled to be 183665006 on ; on the day prior to 75823008 
she deveoped 386661006, 389086002, and 267036007. 399208008 showed b/t lower lobe 409609008. She was started on 
levoflox and ceftriaxone on . When she became 389086002 on NC they brought her in to the ED. . In the ED she was 
386661006 to 102.7, P 109 BP 135/56 R 34 O2 90% on 3L. She was started on vanc and zosyn for broader coverage, 
tylenol, and 2L NS. . The patient reports having sweats and 49727002 before 32485007. She 409586006 of 267036007 
and some upper 161891005. She denies 29857009, URI sx, 16932000, 62315008, or 49650001. Of note she had had a 
271807003 and was given prednisone for 7 days, ending . The 271807003 was speculated to be due to coumadin, but she
was able to be continued on coumadin. Past Medical History: 53741008 s/p stent in 42343007 38341003 PE -  
35489007--on fluoxetine Social History: The patient has been in rehab for the past month. She used to 105529008, 
but has 2 grown daughters living nearby who are involved. They are at the bedside and actively disagreeing about 
the patient's code status and what their mothers's goals of care are. It is unclear if either are HCPs. Family 
History: Doesn't know about siblings health. Children 438949009 and healthy. No medical 55607006. Physical Exam: 
VS: T 99 BP 101/78 P 96 R 23 O2 96% on 100% NRB Gen: 17535004 in mild 271825005. talking in phrases. HEENT: 
418970005, EOMI. MMM, OP : bilateral 48409008 to mid-lung fields, clear anteriorly CV: RRR. nl s1/s2, no M/R/G Abd:
+ BS present; soft, ND/NT. 59614000 in ED Ext: no c/c/e Neuro: A&O x 2. follow commands, MAE. Pertinent Results: 
06:26PM BLOOD 767002-9.6 14089001-3.33* Hgb-8.9* Hct-27.0* MCV-81* MCH-26.7* MCHC-32.9 RDW-14.4 Plt Ct-291 12:17AM 
BLOOD Hct-27.1* 06:26PM BLOOD Neuts-81.0* Lymphs-14.9* Monos-2.0 Eos-1.9 Baso-0.2 06:26PM BLOOD PT-21.7* 
42525009-45.7* INR(PT)-3.4 04:11AM BLOOD Plt Ct-273 06:26PM 365812005-104 24509005-36* Creat-1.4* Na-138 K-4.8 
Cl-104 HCO3-18* AnGap-21* 04:11PM 365812005-103 24509005-22* Creat-1.1 Na-138 K-3.5 Cl-107 HCO3-17* AnGap-18 
06:26PM BLOOD CK(CPK)-56 06:26PM BLOOD CK-MB-NotDone cTropnT-0.01 06:26PM 312472004-8.9 Phos-3.2 Mg-1.7 04:11PM 
312472004-7.9* Phos-3.6 Mg-2.2 06:32PM BLOOD Lactate-3.8* 01:08AM BLOOD Lactate-1.1 01:16AM BLOOD K-3.5 . PORTABLE 
AP CHEST 168537006: The heart size and mediastinal contours are 125112009. No definite 60046008 are seen. There is 
diffusely increased interstitial 128308005 64572001, predominantly in the lower lung zones. No 36118008 seen. The 
osseous structures are stable. A 84089009 is noted. Tiny left 60046008 is noted. IMPRESSION: Diffusely increased 
interstitial 128308005. This appearance is consistent with pulmonary vascular 85804007 superimposed upon chronic 
interstitial changes. . IMPRESSION: AP chest compared to and : Severe progressive interstitial abnormality 
accompanied by pulmonary and mediastinal vascular 85804007 is most likely 267038008, but severe  either infectious 
or drug 262094002 could simulate these findings. Chronic 84089009 unchanged. Brief Hospital Course: 82 yo F with 
53741008, 42343007, 38341003, recent PE ( ), who presents from rehab with PNA and 389086002. . 168734001 revealed 
bilateral 409609008. Patient was started on Zosyn and vancomycin for 233604007. Her fluid status was closely 
monitored given her underlying 42343007. On admission her daughters were in disagreement over her code status and 
her original long 10904000 304253006/DNI status was changed to allow for 52765003 if needed. However, when the 
patient's respiratory status continued to

4844

119